In [1]:
from __future__ import division
from __future__ import unicode_literals
from __future__ import absolute_import
import pandas as pd
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
#from rdkit.Avalon.pyAvalonTools import GetAvalonFP
import rdkit.rdBase
from rdkit.Chem.MACCSkeys import GenMACCSKeys
from rdkit import DataStructs
from rdkit.DataStructs import BitVectToText
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.Chem import Descriptors as Descriptors
from rdkit.Chem.EState import Fingerprinter
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd 
from rdkit.Chem import PandasTools as PandasTools
from rdkit.Chem import Descriptors as Descriptors

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
from rdkit.Chem import DataStructs
from matplotlib import pyplot as plt

In [6]:
model_tox_data = pd.read_csv('../data/Alloriginaldata_19911.csv') 
model_tox_data

,smiles_r,TOXICITY,Source_rank,Source,Data
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI
...,...,...,...,...,...
19906,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox
19907,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19908,CCOC(=O)CN[C@@H](C(=O)N1CC[C@H]1C(=O)NCc1ccc(/...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19909,C=C(C)[C@@]12OC3(/C=C/C=C/CCCCC)O[C@@H]1[C@@H]...,0,16,Reactive Metabolite Mazzolari et al,LivTox


In [9]:
from rdkit import Chem
from rdkit.Chem import Descriptors

# Assuming you have a DataFrame 'df' with a column 'smiles' for SMILES strings

# Initialize counters for each criterion
counters = {
    'invalid_smiles': 0,
    'no_carbon': 0,
    'is_mixture': 0,
    'contains_metals': 0,
    'above_mw_threshold': 0
}

# Function to check exclusion criteria and update counters
def check_criteria(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            counters['invalid_smiles'] += 1
            return False
        if not mol.HasSubstructMatch(Chem.MolFromSmarts('C')):
            counters['no_carbon'] += 1
            return False
        if mol.GetNumAtoms() != Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=False)[0].GetNumAtoms():
            counters['is_mixture'] += 1
            return False
        if any(atom.GetSymbol().isupper() and atom.GetSymbol() not in ['C', 'H', 'N', 'O', 'P', 'S', 'F', 'Cl', 'Br', 'I'] for atom in mol.GetAtoms()):
            counters['contains_metals'] += 1
            return False
        if Descriptors.MolWt(mol) > 1500:
            counters['above_mw_threshold'] += 1
            return False
        return True
    except:
        counters['invalid_smiles'] += 1
        return False

# Apply the criteria checking function to each SMILES string
model_tox_data['include'] = model_tox_data['smiles_r'].apply(check_criteria)

# Filter the DataFrame to include only the molecules that passed all criteria
filtered_df = model_tox_data[model_tox_data['include']]

# Now, print the counters to see how many compounds were excluded for each reason
for criterion, count in counters.items():
    print(f"{criterion}: {count}")

# The result is 'filtered_df' which contains only the molecules meeting the criteria
# You can now drop the 'include' column if you want
filtered_df = filtered_df.drop(columns=['include'])


[19:00:23] Unusual charge on atom 16 number of radical electrons set to zero


invalid_smiles: 0
no_carbon: 1153
is_mixture: 1072
contains_metals: 14
above_mw_threshold: 43


[19:00:24] WARNING: not removing hydrogen atom without neighbors
[19:00:24] WARNING: not removing hydrogen atom without neighbors
[19:00:24] WARNING: not removing hydrogen atom without neighbors
[19:00:24] WARNING: not removing hydrogen atom without neighbors
[19:00:24] WARNING: not removing hydrogen atom without neighbors
[19:00:24] WARNING: not removing hydrogen atom without neighbors


In [11]:
filtered_df

,smiles_r,TOXICITY,Source_rank,Source,Data
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI
2,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI
3,CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI
...,...,...,...,...,...
19906,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox
19907,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19908,CCOC(=O)CN[C@@H](C(=O)N1CC[C@H]1C(=O)NCc1ccc(/...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19909,C=C(C)[C@@]12OC3(/C=C/C=C/CCCCC)O[C@@H]1[C@@H]...,0,16,Reactive Metabolite Mazzolari et al,LivTox


In [12]:
import pandas as pd

from rdkit.Chem import AddHs
from rdkit.Chem import inchi
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.MolStandardize import Standardizer
from rdkit.Chem import MolStandardize
from collections import Counter

import subprocess

from pandarallel import pandarallel
# Initialize pandarallel
pandarallel.initialize(progress_bar=True)

import warnings

# To ignore all deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# To ignore all warnings (not recommended for development, but can be useful in production)
warnings.filterwarnings("ignore")


import logging
from rdkit import RDLogger

# Suppress RDKit warnings and informational messages
RDLogger.DisableLog('rdApp.*')

def standardize_jumpcp(smiles):
    standardizer = Standardizer()
    smiles_original = smiles

    # Read SMILES and convert it to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)
 
    try:
        smiles_clean_counter = Counter()
        mol_dict = {}
        is_finalize = False

        for _ in range(5):
            
            #This solved phosphate oxidation in most cases but introduces a problem for some compounds: eg. geldanamycin where the stable strcutre is returned
            inchi_standardised = Chem.MolToInchi(mol)
            mol = Chem.MolFromInchi(inchi_standardised)
            
            # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
            mol = rdMolStandardize.Cleanup(mol) 
            # if many fragments, get the "parent" (the actual mol we are interested in) 
            mol = rdMolStandardize.FragmentParent(mol)
            # try to neutralize molecule
            uncharger = rdMolStandardize.Uncharger() # annoying, but necessary as no convenience method exists
            
            mol = uncharger.uncharge(mol)# standardize molecules using MolVS and RDKit
            mol = standardizer.charge_parent(mol)
            mol = standardizer.isotope_parent(mol)
            mol = standardizer.stereo_parent(mol)
            
            #Normalize tautomers 
            #Method 1
            normalizer = MolStandardize.tautomer.TautomerCanonicalizer()
            mol = normalizer.canonicalize(mol)
            
            #Method 2
            te = rdMolStandardize.TautomerEnumerator() # idem
            mol = te.Canonicalize(mol)
            
            #Method 3
            mol = standardizer.tautomer_parent(mol)
       
    
            #Final Rules
            mol = standardizer.standardize(mol)
            mol_standardized = mol

            # convert mol object back to SMILES
            smiles_standardized = Chem.MolToSmiles(mol_standardized)

            if smiles == smiles_standardized:
                is_finalize = True
                break

            smiles_clean_counter[smiles_standardized] += 1
            if smiles_standardized not in mol_dict:
                mol_dict[smiles_standardized] = mol_standardized

            smiles = smiles_standardized
            mol = Chem.MolFromSmiles(smiles)

        if not is_finalize:
            # If the standardization process is not finalized, we choose the most common SMILES from the counter
            smiles_standardized = smiles_clean_counter.most_common()[0][0]
            # ... and the corresponding mol object
            #mol_standardized = mol_dict[smiles_standardized]
                
        return smiles_standardized 
    
    except:     
        
        return "Cannot_do"

INFO: Pandarallel will run on 112 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [13]:
# Apply your function with a progress bar
filtered_df["smiles_r"] = filtered_df["smiles_r"].parallel_apply(standardize_jumpcp)
filtered_df

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer enumeration stopped at maximum 1000
Tautomer e

,smiles_r,TOXICITY,Source_rank,Source,Data
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI
2,CCN(CC)Cc1cc(N=c2cc[nH]c3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI
3,CCN(CC)CCCC(C)N=c1cc[nH]c2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI
...,...,...,...,...,...
19906,CC(c1ncncc1F)C(O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox
19907,COCC1OC(=O)c2coc3c2C1(C)C1=C(C3=O)C2CCC(=O)C2(...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19908,CCOC(=O)CNC(C(=O)N1CCC1C(=O)NCc1ccc(C(N)=NO)cc...,0,16,Reactive Metabolite Mazzolari et al,LivTox
19909,C=C(C)C12OC3(C=CC=CCCCCC)OC1C1C4OC4(CO)C(O)C4(...,0,16,Reactive Metabolite Mazzolari et al,LivTox


In [14]:
from dimorphite_dl.dimorphite_dl import DimorphiteDL

def protonate_smiles(smiles):

    dimorphite = DimorphiteDL(min_ph=7.0, max_ph=7.0, pka_precision=0)
    protonated_smiles = dimorphite.protonate(smiles)

    #print("protonated_smiles")

    if len(protonated_smiles) > 0:
                protonated_smiles = protonated_smiles[0]
            
    return protonated_smiles
    
def smiles_to_inchikey(smiles):
 
    try:
        
        # Convert SMILES to a molecule object
        mol = Chem.MolFromSmiles(smiles)
        # Convert the molecule object to an InChI string
        inchi_string = Chem.MolToInchi(mol)
        # Convert the InChI string to an InChIKey
        inchi_key = Chem.inchi.InchiToInchiKey(inchi_string)
        
        return inchi_key
    
    except:     
        
        return "Cannot_do"

In [15]:
filtered_df['InChIKey'] = filtered_df["smiles_r"].parallel_apply(smiles_to_inchikey)
filtered_df['InChIKey14'] = filtered_df["InChIKey"].str[:14]

In [16]:
filtered_df['protonated_smiles_r'] = filtered_df["smiles_r"].parallel_apply(protonate_smiles)
filtered_df

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI,GSNOZLZNQMLSKJ-UHFFFAOYSA-N,GSNOZLZNQMLSKJ,CCN(CC)c1cc(C)nc2ncnn12
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI,QTGAJCQTLIRCFL-UHFFFAOYSA-N,QTGAJCQTLIRCFL,CC[NH+](CC)CC(=O)Oc1ccc(NC(C)=O)cc1
2,CCN(CC)Cc1cc(N=c2cc[nH]c3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI,OVCDSSHSILBFBN-UHFFFAOYSA-N,OVCDSSHSILBFBN,CC[NH+](CC)Cc1cc(N=c2cc[nH]c3cc(Cl)ccc23)ccc1O
3,CCN(CC)CCCC(C)N=c1cc[nH]c2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI,WHTVZRBIWZFKQO-UHFFFAOYSA-N,WHTVZRBIWZFKQO,CC[NH+](CC)CCCC(C)N=c1cc[nH]c2cc(Cl)ccc12
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI,SAADBVWGJQAEFS-UHFFFAOYSA-N,SAADBVWGJQAEFS,CC[NH+](CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21
...,...,...,...,...,...,...,...,...
19906,CC(c1ncncc1F)C(O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox,BCEHBSKCWLPMDN-UHFFFAOYSA-N,BCEHBSKCWLPMDN,CC(c1ncncc1F)C(O)(Cn1cncn1)c1ccc(F)cc1F
19907,COCC1OC(=O)c2coc3c2C1(C)C1=C(C3=O)C2CCC(=O)C2(...,0,16,Reactive Metabolite Mazzolari et al,LivTox,QDLHCMPXEPAAMD-UHFFFAOYSA-N,QDLHCMPXEPAAMD,COCC1OC(=O)c2coc3c2C1(C)C1=C(C3=O)C2CCC(=O)C2(...
19908,CCOC(=O)CNC(C(=O)N1CCC1C(=O)NCc1ccc(C(N)=NO)cc...,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZXIBCJHYVWYIKI-UHFFFAOYSA-N,ZXIBCJHYVWYIKI,CCOC(=O)C[NH2+]C(C(=O)[NH+]1CCC1C(=O)NCc1ccc(C...
19909,C=C(C)C12OC3(C=CC=CCCCCC)OC1C1C4OC4(CO)C(O)C4(...,0,16,Reactive Metabolite Mazzolari et al,LivTox,CGSGRJNIABXQJQ-UHFFFAOYSA-N,CGSGRJNIABXQJQ,C=C(C)C12OC3(C=CC=CCCCCC)OC1C1C4OC4(CO)C(O)C4(...


In [17]:
filtered_df

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r
0,CCN(CC)c1cc(C)nc2ncnn12,1,1,DILIst Classification,DILI,GSNOZLZNQMLSKJ-UHFFFAOYSA-N,GSNOZLZNQMLSKJ,CCN(CC)c1cc(C)nc2ncnn12
1,CCN(CC)CC(=O)Oc1ccc(NC(C)=O)cc1,1,1,DILIst Classification,DILI,QTGAJCQTLIRCFL-UHFFFAOYSA-N,QTGAJCQTLIRCFL,CC[NH+](CC)CC(=O)Oc1ccc(NC(C)=O)cc1
2,CCN(CC)Cc1cc(N=c2cc[nH]c3cc(Cl)ccc23)ccc1O,1,1,DILIst Classification,DILI,OVCDSSHSILBFBN-UHFFFAOYSA-N,OVCDSSHSILBFBN,CC[NH+](CC)Cc1cc(N=c2cc[nH]c3cc(Cl)ccc23)ccc1O
3,CCN(CC)CCCC(C)N=c1cc[nH]c2cc(Cl)ccc12,1,1,DILIst Classification Oral,DILI,WHTVZRBIWZFKQO-UHFFFAOYSA-N,WHTVZRBIWZFKQO,CC[NH+](CC)CCCC(C)N=c1cc[nH]c2cc(Cl)ccc12
4,CCN(CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,1,1,DILIst Classification Oral,DILI,SAADBVWGJQAEFS-UHFFFAOYSA-N,SAADBVWGJQAEFS,CC[NH+](CC)CCN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21
...,...,...,...,...,...,...,...,...
19906,CC(c1ncncc1F)C(O)(Cn1cncn1)c1ccc(F)cc1F,0,16,Reactive Metabolite Mazzolari et al,LivTox,BCEHBSKCWLPMDN-UHFFFAOYSA-N,BCEHBSKCWLPMDN,CC(c1ncncc1F)C(O)(Cn1cncn1)c1ccc(F)cc1F
19907,COCC1OC(=O)c2coc3c2C1(C)C1=C(C3=O)C2CCC(=O)C2(...,0,16,Reactive Metabolite Mazzolari et al,LivTox,QDLHCMPXEPAAMD-UHFFFAOYSA-N,QDLHCMPXEPAAMD,COCC1OC(=O)c2coc3c2C1(C)C1=C(C3=O)C2CCC(=O)C2(...
19908,CCOC(=O)CNC(C(=O)N1CCC1C(=O)NCc1ccc(C(N)=NO)cc...,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZXIBCJHYVWYIKI-UHFFFAOYSA-N,ZXIBCJHYVWYIKI,CCOC(=O)C[NH2+]C(C(=O)[NH+]1CCC1C(=O)NCc1ccc(C...
19909,C=C(C)C12OC3(C=CC=CCCCCC)OC1C1C4OC4(CO)C(O)C4(...,0,16,Reactive Metabolite Mazzolari et al,LivTox,CGSGRJNIABXQJQ-UHFFFAOYSA-N,CGSGRJNIABXQJQ,C=C(C)C12OC3(C=CC=CCCCCC)OC1C1C4OC4(CO)C(O)C4(...


In [85]:
import pandas as pd

# Assuming your DataFrame is named 'model_tox_data' and has been properly loaded.

# Step 1: Group by 'Source'. We'll process each group separately.
grouped = filtered_df.groupby('Source_rank')

# Placeholder for processed groups
processed_groups = []

for _, group in grouped:
    # Step 2: Sort within each group by 'TOXICITY' in descending order (1s before 0s)
    # and by 'InChIKey14' to ensure consistency in duplicate removal
    sorted_group = group.sort_values(by=['TOXICITY', 'InChIKey14'], ascending=[False, True])
    
    # Step 3: Drop duplicates based on 'InChIKey14', keeping the first (the toxic one if it exists)
    deduped_group = sorted_group.drop_duplicates(subset='InChIKey14', keep='first')
    
    # Collecting processed groups
    processed_groups.append(deduped_group)

# Step 4: Combine the processed groups back into a single DataFrame
filtered_df_2 = pd.concat(processed_groups, ignore_index=True)

# Display or return the processed DataFrame
filtered_df_2 = filtered_df_2[filtered_df_2["smiles_r"]!="Cannot_do"]
filtered_df_2 = filtered_df_2[filtered_df_2["InChIKey"]!="Cannot_do"]
filtered_df_2 = filtered_df_2[filtered_df_2["protonated_smiles_r"]!="Cannot_do"]
filtered_df_2

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r
0,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,1,1,DILIst Classification Oral,DILI,AAKJLRGGTJKAMG-UHFFFAOYSA-N,AAKJLRGGTJKAMG,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1
1,Oc1c(Cl)cc(Cl)c(Cl)c1Cc1c(O)c(Cl)cc(Cl)c1Cl,1,1,DILIst Classification Topical,DILI,ACGUYXCXAPNIKK-UHFFFAOYSA-N,ACGUYXCXAPNIKK,Oc1c(Cl)cc(Cl)c(Cl)c1Cc1c(O)c(Cl)cc(Cl)c1Cl
2,O=C(O)C(S)C(S)C(=O)O,1,1,DILIst Classification Oral,DILI,ACTRVOBWPAIOHC-UHFFFAOYSA-N,ACTRVOBWPAIOHC,O=C([O-])C(S)C(S)C(=O)[O-]
3,CCCCC(=O)N(Cc1ccc(-c2ccccc2-c2nn[nH]n2)cc1)C(C...,1,1,DILIst Classification Oral,DILI,ACWBQPMHZXGDFX-UHFFFAOYSA-N,ACWBQPMHZXGDFX,CCCCC(=O)[NH+](Cc1ccc(-c2ccccc2-c2nn[nH]n2)cc1...
4,COc1cc2c(cc1OC)C(=O)C(CC1CCN(Cc3ccccc3)CC1)C2,1,1,DILIst Classification Oral,DILI,ADEBPBSSDYVVLD-UHFFFAOYSA-N,ADEBPBSSDYVVLD,COc1cc2c(cc1OC)C(=O)C(CC1CC[NH+](Cc3ccccc3)CC1)C2
...,...,...,...,...,...,...,...,...
17430,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZLHVIYHWWQYJID-UHFFFAOYSA-N,ZLHVIYHWWQYJID,O=C1CC(c2ccc(O)cc2)Oc2ccccc21
17431,CCCCSP(=O)(SCCCC)SCCCC,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZOKXUAHZSKEQSS-UHFFFAOYSA-N,ZOKXUAHZSKEQSS,CCCCSP(=O)(SCCCC)SCCCC
17432,O=C1CC(c2ccccc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZONYXWQDUYMKFB-UHFFFAOYSA-N,ZONYXWQDUYMKFB,O=C1CC(c2ccccc2)Oc2ccccc21
17433,O=C(O)CCC(=O)c1ccc(-c2ccccc2)cc1,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZPAKPRAICRBAOD-UHFFFAOYSA-N,ZPAKPRAICRBAOD,O=C([O-])CCC(=O)c1ccc(-c2ccccc2)cc1


In [86]:
train_data_tox = filtered_df_2[filtered_df_2["Data"]!="DILI"]
#train_data_tox = train_data_tox[["smiles_r","TOXICITY"]].reset_index(drop=True)
train_data_tox["Data"] = "LivTox"
train_data_tox

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r
1077,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AHLBNYSZXLDEJQ-UHFFFAOYSA-N,AHLBNYSZXLDEJQ,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...
1078,CCN(CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3ccc(F...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,ALVHFWNGOPERPY-UHFFFAOYSA-N,ALVHFWNGOPERPY,CC[NH+](CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3c...
1079,CN=C(NC#N)NCCSCc1[nH]cnc1C,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AQIXAKUUQRKLND-UHFFFAOYSA-N,AQIXAKUUQRKLND,C[NH+]=C(NC#N)NCCSCc1[nH]cnc1C
1080,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)O)C1OC1OC(C(=O)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AVJBPWGFOQAPRH-UHFFFAOYSA-N,AVJBPWGFOQAPRH,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)[O-])C1OC1OC(C(...
1081,O=C(O)c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,BOFQWVMAQOTZIW-UHFFFAOYSA-N,BOFQWVMAQOTZIW,O=C([O-])c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1
...,...,...,...,...,...,...,...,...
17430,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZLHVIYHWWQYJID-UHFFFAOYSA-N,ZLHVIYHWWQYJID,O=C1CC(c2ccc(O)cc2)Oc2ccccc21
17431,CCCCSP(=O)(SCCCC)SCCCC,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZOKXUAHZSKEQSS-UHFFFAOYSA-N,ZOKXUAHZSKEQSS,CCCCSP(=O)(SCCCC)SCCCC
17432,O=C1CC(c2ccccc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZONYXWQDUYMKFB-UHFFFAOYSA-N,ZONYXWQDUYMKFB,O=C1CC(c2ccccc2)Oc2ccccc21
17433,O=C(O)CCC(=O)c1ccc(-c2ccccc2)cc1,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZPAKPRAICRBAOD-UHFFFAOYSA-N,ZPAKPRAICRBAOD,O=C([O-])CCC(=O)c1ccc(-c2ccccc2)cc1


In [87]:
outer_test_data_tox = filtered_df_2[filtered_df_2["Data"]=="DILI"]
#outer_test_data_tox=outer_test_data_tox[["smiles_r","TOXICITY"]].reset_index(drop=True)
outer_test_data_tox = outer_test_data_tox.sort_values(["TOXICITY"], ascending=False)
outer_test_data_tox = outer_test_data_tox.drop_duplicates(subset=["InChIKey14"], keep="first").reset_index(drop=True)
outer_test_data_tox = outer_test_data_tox[outer_test_data_tox["smiles_r"]!="Cannot_do"]
outer_test_data_tox["Data"] = "DILI"
outer_test_data_tox

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r
0,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,1,1,DILIst Classification Oral,DILI,AAKJLRGGTJKAMG-UHFFFAOYSA-N,AAKJLRGGTJKAMG,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1
1,COC1(NC(=O)CSC(F)F)C(=O)N2C(C(=O)O)=C(CSc3nnnn...,1,1,DILIst Classification,DILI,UHRBTBZOWWGKMK-UHFFFAOYSA-N,UHRBTBZOWWGKMK,COC1(NC(=O)CSC(F)F)C(=O)[NH+]2C(C(=O)[O-])=C(C...
2,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)N1CCCC1,1,1,DILIst Classification,DILI,UIEATEWHFDRYRU-UHFFFAOYSA-N,UIEATEWHFDRYRU,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)[NH+]1CCCC1
3,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)...,1,1,DILIst Classification oral,DILI,UIOFUWFRIANQPC-UHFFFAOYSA-N,UIOFUWFRIANQPC,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)[NH+]2C1S...
4,CC1OC1[P](=O)(=O)O,1,1,DILIst Classification Oral,DILI,UJNUDOLLRRCQDH-UHFFFAOYSA-N,UJNUDOLLRRCQDH,CC1OC1[P](=O)(=O)O
...,...,...,...,...,...,...,...,...
1106,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,1,DILIst Classification,DILI,KYRVNWMVYQXFEU-UHFFFAOYSA-N,KYRVNWMVYQXFEU,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1
1107,CC1CNc2c(cccc2S(=O)(=O)NC(CCCN=C(N)N)C(=O)N2CC...,0,1,DILIst Classification Intravenous,DILI,KXNPVXPOPUZYGB-UHFFFAOYSA-N,KXNPVXPOPUZYGB,CC1CNc2c(cccc2S(=O)(=O)NC(CCC[NH+]=C(N)N)C(=O)...
1108,CCC(c1cccc(O)c1)C(C)CN(C)C,0,1,DILIst Classification Oral,DILI,KWTWDQCKEHXFFR-UHFFFAOYSA-N,KWTWDQCKEHXFFR,CCC(c1cccc(O)c1)C(C)C[NH+](C)C
1109,CC(N)Cc1ccccc1,0,1,DILIst Classification Oral,DILI,KWTSXDURSIMDCE-UHFFFAOYSA-N,KWTSXDURSIMDCE,CC([NH3+])Cc1ccccc1


In [88]:
outer_test_data_tox.InChIKey14.nunique()

1111

In [89]:
outer_test_data_tox.protonated_smiles_r.nunique()

1111

In [90]:
outer_test_data_tox.to_csv("DILI_Goldstandard_1111.csv", index=False)

In [91]:
model_tox_data = pd.concat([train_data_tox, outer_test_data_tox]).reset_index(drop=True)
model_tox_data

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r
0,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AHLBNYSZXLDEJQ-UHFFFAOYSA-N,AHLBNYSZXLDEJQ,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...
1,CCN(CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3ccc(F...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,ALVHFWNGOPERPY-UHFFFAOYSA-N,ALVHFWNGOPERPY,CC[NH+](CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3c...
2,CN=C(NC#N)NCCSCc1[nH]cnc1C,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AQIXAKUUQRKLND-UHFFFAOYSA-N,AQIXAKUUQRKLND,C[NH+]=C(NC#N)NCCSCc1[nH]cnc1C
3,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)O)C1OC1OC(C(=O)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AVJBPWGFOQAPRH-UHFFFAOYSA-N,AVJBPWGFOQAPRH,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)[O-])C1OC1OC(C(...
4,O=C(O)c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,BOFQWVMAQOTZIW-UHFFFAOYSA-N,BOFQWVMAQOTZIW,O=C([O-])c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1
...,...,...,...,...,...,...,...,...
17316,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,1,DILIst Classification,DILI,KYRVNWMVYQXFEU-UHFFFAOYSA-N,KYRVNWMVYQXFEU,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1
17317,CC1CNc2c(cccc2S(=O)(=O)NC(CCCN=C(N)N)C(=O)N2CC...,0,1,DILIst Classification Intravenous,DILI,KXNPVXPOPUZYGB-UHFFFAOYSA-N,KXNPVXPOPUZYGB,CC1CNc2c(cccc2S(=O)(=O)NC(CCC[NH+]=C(N)N)C(=O)...
17318,CCC(c1cccc(O)c1)C(C)CN(C)C,0,1,DILIst Classification Oral,DILI,KWTWDQCKEHXFFR-UHFFFAOYSA-N,KWTWDQCKEHXFFR,CCC(c1cccc(O)c1)C(C)C[NH+](C)C
17319,CC(N)Cc1ccccc1,0,1,DILIst Classification Oral,DILI,KWTSXDURSIMDCE-UHFFFAOYSA-N,KWTSXDURSIMDCE,CC([NH3+])Cc1ccccc1


In [92]:
def MorganFingerprint(s):
    x = Chem.MolFromSmiles(s)
    return (AllChem.GetMorganFingerprintAsBitVect(x,2,2048))

In [93]:
MorganFingerprint_array = np.stack(model_tox_data['protonated_smiles_r'].apply(MorganFingerprint))

Morgan_fingerprint_collection = []
for x in np.arange(MorganFingerprint_array.shape[1]):
    x = "Mfp"+str(x)
    Morgan_fingerprint_collection.append(x)

Morgan_fingerprint_table = pd.DataFrame(MorganFingerprint_array, columns=Morgan_fingerprint_collection)

Morgan_fingerprint_table

,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,Mfp5,Mfp6,Mfp7,Mfp8,Mfp9,...,Mfp2038,Mfp2039,Mfp2040,Mfp2041,Mfp2042,Mfp2043,Mfp2044,Mfp2045,Mfp2046,Mfp2047
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17317,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17318,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17319,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
def MACCSKeysFingerprint(s):
    x = Chem.MolFromSmiles(s)
    return (AllChem.GetMACCSKeysFingerprint(x))

MACCSfingerprint_array = np.stack(model_tox_data['protonated_smiles_r'].apply(MACCSKeysFingerprint))

MACCS_collection = []
for x in np.arange(MACCSfingerprint_array.shape[1]):
    x = "MACCS"+str(x)
    MACCS_collection.append(x)

MACCSfingerprint_table = pd.DataFrame(MACCSfingerprint_array, columns=MACCS_collection)

MACCSfingerprint_table

,MACCS0,MACCS1,MACCS2,MACCS3,MACCS4,MACCS5,MACCS6,MACCS7,MACCS8,MACCS9,...,MACCS157,MACCS158,MACCS159,MACCS160,MACCS161,MACCS162,MACCS163,MACCS164,MACCS165,MACCS166
0,0,0,0,0,0,0,0,0,1,0,...,1,1,1,1,1,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
17317,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
17318,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
17319,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0


In [95]:
2048+166

2214

In [96]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 112 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [97]:
from rdkit import Chem
from mordred import Calculator, descriptors

# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

print(len(calc.descriptors))

Ser_Mol = model_tox_data['protonated_smiles_r'].apply(Chem.MolFromSmiles)

# as pandas
Mordred_table=  calc.pandas(Ser_Mol)
Mordred_table = Mordred_table.astype('float')
#Mordred_table['smiles_r'] = model_tox_data['smiles_r']
Mordred_table

1613


100%|████████████████████████████████████| 17321/17321 [01:27<00:00, 196.95it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,25.064976,20.343867,0.0,0.0,42.333994,2.413016,4.826033,42.333994,1.209543,4.416495,...,10.066499,70.246324,495.392374,5.629459,4600.0,41.0,154.0,167.0,13.027778,8.500000
1,22.184178,18.708070,0.0,1.0,36.668582,2.495150,4.810199,36.668582,1.264434,4.287622,...,10.247184,80.736166,399.219081,7.003844,2452.0,46.0,150.0,178.0,10.972222,6.527778
2,12.089765,11.136291,0.0,3.0,21.154703,2.257483,4.368546,21.154703,1.244394,3.701955,...,9.034200,61.513019,253.122992,7.444794,664.0,18.0,74.0,80.0,6.083333,4.194444
3,23.615330,20.818875,2.0,0.0,36.633186,2.486275,4.972550,36.633186,1.181716,4.337450,...,10.438664,67.002785,475.064287,9.135852,2440.0,53.0,160.0,187.0,14.895833,6.708333
4,22.074789,17.282799,1.0,0.0,36.545688,2.493808,4.866787,36.545688,1.305203,4.272606,...,10.291908,77.536823,372.098980,8.859500,1976.0,46.0,150.0,179.0,8.611111,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,16.360915,13.772089,0.0,0.0,27.389536,2.437253,4.767349,27.389536,1.304264,3.982281,...,9.858333,70.378545,301.052112,9.407879,994.0,29.0,110.0,129.0,6.527778,4.666667
17317,26.942267,21.807837,1.0,4.0,43.075270,2.473743,4.947486,43.075270,1.230722,4.465095,...,10.488075,71.463013,509.254066,7.072973,3704.0,57.0,180.0,209.0,13.784722,7.625000
17318,11.670387,10.989759,0.0,1.0,18.713920,2.321693,4.643385,18.713920,1.169620,3.652975,...,9.353834,47.590867,222.185241,5.554631,448.0,21.0,74.0,82.0,7.055556,3.722222
17319,7.289847,6.783462,0.0,1.0,12.066822,2.175328,4.350655,12.066822,1.206682,3.188073,...,8.564268,38.519688,136.112076,5.671336,126.0,9.0,44.0,46.0,3.722222,2.333333


In [98]:
Mordred_table = Mordred_table.dropna(axis='columns')
Mordred_table

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,25.064976,20.343867,0.0,0.0,42.333994,2.413016,4.826033,42.333994,1.209543,4.416495,...,0.000000,10.066499,70.246324,495.392374,5.629459,4600.0,41.0,154.0,167.0,8.500000
1,22.184178,18.708070,0.0,1.0,36.668582,2.495150,4.810199,36.668582,1.264434,4.287622,...,7.703008,10.247184,80.736166,399.219081,7.003844,2452.0,46.0,150.0,178.0,6.527778
2,12.089765,11.136291,0.0,3.0,21.154703,2.257483,4.368546,21.154703,1.244394,3.701955,...,6.529419,9.034200,61.513019,253.122992,7.444794,664.0,18.0,74.0,80.0,4.194444
3,23.615330,20.818875,2.0,0.0,36.633186,2.486275,4.972550,36.633186,1.181716,4.337450,...,0.000000,10.438664,67.002785,475.064287,9.135852,2440.0,53.0,160.0,187.0,6.708333
4,22.074789,17.282799,1.0,0.0,36.545688,2.493808,4.866787,36.545688,1.305203,4.272606,...,6.861711,10.291908,77.536823,372.098980,8.859500,1976.0,46.0,150.0,179.0,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,16.360915,13.772089,0.0,0.0,27.389536,2.437253,4.767349,27.389536,1.304264,3.982281,...,7.285507,9.858333,70.378545,301.052112,9.407879,994.0,29.0,110.0,129.0,4.666667
17317,26.942267,21.807837,1.0,4.0,43.075270,2.473743,4.947486,43.075270,1.230722,4.465095,...,0.000000,10.488075,71.463013,509.254066,7.072973,3704.0,57.0,180.0,209.0,7.625000
17318,11.670387,10.989759,0.0,1.0,18.713920,2.321693,4.643385,18.713920,1.169620,3.652975,...,0.000000,9.353834,47.590867,222.185241,5.554631,448.0,21.0,74.0,82.0,3.722222
17319,7.289847,6.783462,0.0,1.0,12.066822,2.175328,4.350655,12.066822,1.206682,3.188073,...,0.000000,8.564268,38.519688,136.112076,5.671336,126.0,9.0,44.0,46.0,2.333333


In [99]:
def get_num_charged_atoms_neg(mol):
    mol_h = Chem.AddHs(mol)
    Chem.rdPartialCharges.ComputeGasteigerCharges(mol_h)
     
    positive = 0
    negative = 0
     
    for atom in mol_h.GetAtoms():
        if float(atom.GetProp('_GasteigerCharge')) <= 0:
            negative += 1
     
    return negative

def get_num_charged_atoms_pos(mol):
    mol_h = Chem.AddHs(mol)
    Chem.rdPartialCharges.ComputeGasteigerCharges(mol_h)
     
    positive = 0
    negative = 0
     
    for atom in mol_h.GetAtoms():
        if float(atom.GetProp('_GasteigerCharge')) >= 0:
            positive += 1
    return positive

def get_assembled_ring(mol):
    ring_info = mol.GetRingInfo()
    num_ring = ring_info.NumRings()
    ring_atoms = ring_info.AtomRings()
    num_assembled = 0
     
    for i in range(num_ring):
        for j in range(i+1, num_ring):
            x = set(ring_atoms[i])
            y = set(ring_atoms[j])
            if not x.intersection(y): # 2つの環が縮環でない場合に
                for x_id in x:
                    x_atom = mol.GetAtomWithIdx(x_id)
                    neighbors = [k.GetIdx() for k in x_atom.GetNeighbors()]
                    for x_n in neighbors:
                        if x_n in y: # 環同士を繋ぐ結合があるか否か
                            num_assembled += 1
     
    return num_assembled

def get_num_stereocenters(mol):
    return AllChem.CalcNumAtomStereoCenters(mol) + AllChem.CalcNumUnspecifiedAtomStereoCenters(mol)

def calc_descriptors(dataframe):
    mols = dataframe.protonated_smiles_r.apply(Chem.MolFromSmiles)
    descr = []
    for m in mols:
        descr.append([ Descriptors.TPSA(m),
               Descriptors.NumRotatableBonds(m),
               AllChem.CalcNumRings(m),
               Descriptors.NumAromaticRings(m),
               Descriptors.NumHAcceptors(m),
               Descriptors.NumHDonors(m),
               Descriptors.FractionCSP3(m),
               Descriptors.MolLogP(m) ,
               Descriptors.NHOHCount(m),
               Descriptors.NOCount(m),
               Descriptors.NumHeteroatoms(m),
               get_num_charged_atoms_pos(m),
               get_num_charged_atoms_neg(m),
               get_assembled_ring(m),
               get_num_stereocenters(m)])
    descr = np.asarray(descr)
    return(descr)

descs = [ 'PSA', 'n_rot_bonds', 'n_rings', 'n_ar_rings',
         'n_HBA', 'n_HBD', 'Fsp3', 'logP', 'NHOHCount', 'NOCount', 'NumHeteroatoms',
        'n_positive', '_n_negative', 'n_ring_asmbl', 'n_stereo']
a=calc_descriptors(model_tox_data)
descdf=pd.DataFrame(a, columns=descs)
descdf_approved=descdf.reset_index(drop=True)
descdf_approved

,PSA,n_rot_bonds,n_rings,n_ar_rings,n_HBA,n_HBD,Fsp3,logP,NHOHCount,NOCount,NumHeteroatoms,n_positive,_n_negative,n_ring_asmbl,n_stereo
0,81.70,23.0,1.0,0.0,5.0,1.0,0.896552,6.88190,1.0,6.0,6.0,61.0,27.0,0.0,8.0
1,85.68,7.0,3.0,3.0,2.0,4.0,0.363636,2.68404,4.0,6.0,7.0,42.0,15.0,0.0,0.0
2,90.50,5.0,1.0,1.0,3.0,4.0,0.500000,-1.32200,4.0,6.0,7.0,26.0,8.0,0.0,0.0
3,264.50,7.0,2.0,0.0,15.0,6.0,0.857143,-7.01130,6.0,16.0,17.0,36.0,16.0,0.0,20.0
4,111.30,4.0,4.0,4.0,7.0,2.0,0.000000,2.37600,2.0,7.0,7.0,22.0,20.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,87.31,2.0,3.0,3.0,4.0,2.0,0.071429,2.45560,2.0,6.0,7.0,19.0,13.0,0.0,0.0
17317,185.85,9.0,3.0,1.0,6.0,6.0,0.608696,-3.96830,8.0,11.0,12.0,49.0,23.0,0.0,10.0
17318,24.67,5.0,1.0,1.0,1.0,2.0,0.571429,1.66640,2.0,2.0,2.0,29.0,11.0,0.0,4.0
17319,27.64,2.0,1.0,1.0,0.0,1.0,0.333333,0.85950,3.0,1.0,1.0,16.0,8.0,0.0,2.0


In [100]:
tox_model_data= pd.concat([model_tox_data, Morgan_fingerprint_table, MACCSfingerprint_table, descdf_approved, Mordred_table], axis=1)
tox_model_data

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AHLBNYSZXLDEJQ-UHFFFAOYSA-N,AHLBNYSZXLDEJQ,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,0,1,...,0.000000,10.066499,70.246324,495.392374,5.629459,4600.0,41.0,154.0,167.0,8.500000
1,CCN(CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3ccc(F...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,ALVHFWNGOPERPY-UHFFFAOYSA-N,ALVHFWNGOPERPY,CC[NH+](CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3c...,0,0,...,7.703008,10.247184,80.736166,399.219081,7.003844,2452.0,46.0,150.0,178.0,6.527778
2,CN=C(NC#N)NCCSCc1[nH]cnc1C,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AQIXAKUUQRKLND-UHFFFAOYSA-N,AQIXAKUUQRKLND,C[NH+]=C(NC#N)NCCSCc1[nH]cnc1C,0,0,...,6.529419,9.034200,61.513019,253.122992,7.444794,664.0,18.0,74.0,80.0,4.194444
3,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)O)C1OC1OC(C(=O)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AVJBPWGFOQAPRH-UHFFFAOYSA-N,AVJBPWGFOQAPRH,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)[O-])C1OC1OC(C(...,0,0,...,0.000000,10.438664,67.002785,475.064287,9.135852,2440.0,53.0,160.0,187.0,6.708333
4,O=C(O)c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,BOFQWVMAQOTZIW-UHFFFAOYSA-N,BOFQWVMAQOTZIW,O=C([O-])c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,0,0,...,6.861711,10.291908,77.536823,372.098980,8.859500,1976.0,46.0,150.0,179.0,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,1,DILIst Classification,DILI,KYRVNWMVYQXFEU-UHFFFAOYSA-N,KYRVNWMVYQXFEU,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,0,...,7.285507,9.858333,70.378545,301.052112,9.407879,994.0,29.0,110.0,129.0,4.666667
17317,CC1CNc2c(cccc2S(=O)(=O)NC(CCCN=C(N)N)C(=O)N2CC...,0,1,DILIst Classification Intravenous,DILI,KXNPVXPOPUZYGB-UHFFFAOYSA-N,KXNPVXPOPUZYGB,CC1CNc2c(cccc2S(=O)(=O)NC(CCC[NH+]=C(N)N)C(=O)...,0,1,...,0.000000,10.488075,71.463013,509.254066,7.072973,3704.0,57.0,180.0,209.0,7.625000
17318,CCC(c1cccc(O)c1)C(C)CN(C)C,0,1,DILIst Classification Oral,DILI,KWTWDQCKEHXFFR-UHFFFAOYSA-N,KWTWDQCKEHXFFR,CCC(c1cccc(O)c1)C(C)C[NH+](C)C,0,1,...,0.000000,9.353834,47.590867,222.185241,5.554631,448.0,21.0,74.0,82.0,3.722222
17319,CC(N)Cc1ccccc1,0,1,DILIst Classification Oral,DILI,KWTSXDURSIMDCE-UHFFFAOYSA-N,KWTSXDURSIMDCE,CC([NH3+])Cc1ccccc1,0,1,...,0.000000,8.564268,38.519688,136.112076,5.671336,126.0,9.0,44.0,46.0,2.333333


In [101]:
test_data_features= tox_model_data[tox_model_data['Data']=="DILI"].reset_index(drop=True)

In [102]:
train_data_features= tox_model_data[tox_model_data['Data']=="LivTox"].reset_index(drop=True)

In [103]:
train_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AHLBNYSZXLDEJQ-UHFFFAOYSA-N,AHLBNYSZXLDEJQ,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,0,1,...,0.000000,10.066499,70.246324,495.392374,5.629459,4600.0,41.0,154.0,167.0,8.500000
1,CCN(CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3ccc(F...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,ALVHFWNGOPERPY-UHFFFAOYSA-N,ALVHFWNGOPERPY,CC[NH+](CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3c...,0,0,...,7.703008,10.247184,80.736166,399.219081,7.003844,2452.0,46.0,150.0,178.0,6.527778
2,CN=C(NC#N)NCCSCc1[nH]cnc1C,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AQIXAKUUQRKLND-UHFFFAOYSA-N,AQIXAKUUQRKLND,C[NH+]=C(NC#N)NCCSCc1[nH]cnc1C,0,0,...,6.529419,9.034200,61.513019,253.122992,7.444794,664.0,18.0,74.0,80.0,4.194444
3,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)O)C1OC1OC(C(=O)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AVJBPWGFOQAPRH-UHFFFAOYSA-N,AVJBPWGFOQAPRH,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)[O-])C1OC1OC(C(...,0,0,...,0.000000,10.438664,67.002785,475.064287,9.135852,2440.0,53.0,160.0,187.0,6.708333
4,O=C(O)c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,BOFQWVMAQOTZIW-UHFFFAOYSA-N,BOFQWVMAQOTZIW,O=C([O-])c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,0,0,...,6.861711,10.291908,77.536823,372.098980,8.859500,1976.0,46.0,150.0,179.0,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16205,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZLHVIYHWWQYJID-UHFFFAOYSA-N,ZLHVIYHWWQYJID,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,0,...,0.000000,9.860110,51.351894,240.078644,8.002621,598.0,28.0,96.0,113.0,3.916667
16206,CCCCSP(=O)(SCCCC)SCCCC,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZOKXUAHZSKEQSS-UHFFFAOYSA-N,ZOKXUAHZSKEQSS,CCCCSP(=O)(SCCCC)SCCCC,0,0,...,0.000000,8.981304,47.595574,314.096165,7.138549,616.0,18.0,68.0,70.0,4.375000
16207,O=C1CC(c2ccccc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZONYXWQDUYMKFB-UHFFFAOYSA-N,ZONYXWQDUYMKFB,O=C1CC(c2ccccc2)Oc2ccccc21,0,0,...,0.000000,9.786336,50.012654,224.083730,7.727025,500.0,26.0,90.0,106.0,3.750000
16208,O=C(O)CCC(=O)c1ccc(-c2ccccc2)cc1,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZPAKPRAICRBAOD-UHFFFAOYSA-N,ZPAKPRAICRBAOD,O=C([O-])CCC(=O)c1ccc(-c2ccccc2)cc1,0,0,...,0.000000,9.556834,51.655875,253.087018,7.908969,808.0,25.0,92.0,103.0,4.305556


In [104]:
test_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,1,1,DILIst Classification Oral,DILI,AAKJLRGGTJKAMG-UHFFFAOYSA-N,AAKJLRGGTJKAMG,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,0,0,...,0.000000,10.116298,64.016835,393.168856,7.560940,2396.0,43.0,142.0,163.0,6.833333
1,COC1(NC(=O)CSC(F)F)C(=O)N2C(C(=O)O)=C(CSc3nnnn...,1,1,DILIst Classification,DILI,UHRBTBZOWWGKMK-UHFFFAOYSA-N,UHRBTBZOWWGKMK,COC1(NC(=O)CSC(F)F)C(=O)[NH+]2C(C(=O)[O-])=C(C...,0,1,...,6.555357,10.733545,82.269503,496.064645,9.921293,3113.0,51.0,166.0,200.0,7.250000
2,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)N1CCCC1,1,1,DILIst Classification,DILI,UIEATEWHFDRYRU-UHFFFAOYSA-N,UIEATEWHFDRYRU,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)[NH+]1CCCC1,0,1,...,6.293419,9.845276,74.440588,367.274390,5.923780,1895.0,34.0,132.0,148.0,6.138889
3,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)...,1,1,DILIst Classification oral,DILI,UIOFUWFRIANQPC-UHFFFAOYSA-N,UIOFUWFRIANQPC,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)[NH+]2C1S...,0,0,...,7.853605,10.768843,83.380114,453.056148,9.639493,2402.0,53.0,170.0,211.0,6.291667
4,CC1OC1[P](=O)(=O)O,1,1,DILIst Classification Oral,DILI,UJNUDOLLRRCQDH-UHFFFAOYSA-N,UJNUDOLLRRCQDH,CC1OC1[P](=O)(=O)O,0,0,...,7.753194,9.180603,57.029675,137.000370,9.785741,61.0,7.0,42.0,48.0,1.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,1,DILIst Classification,DILI,KYRVNWMVYQXFEU-UHFFFAOYSA-N,KYRVNWMVYQXFEU,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,0,...,7.285507,9.858333,70.378545,301.052112,9.407879,994.0,29.0,110.0,129.0,4.666667
1107,CC1CNc2c(cccc2S(=O)(=O)NC(CCCN=C(N)N)C(=O)N2CC...,0,1,DILIst Classification Intravenous,DILI,KXNPVXPOPUZYGB-UHFFFAOYSA-N,KXNPVXPOPUZYGB,CC1CNc2c(cccc2S(=O)(=O)NC(CCC[NH+]=C(N)N)C(=O)...,0,1,...,0.000000,10.488075,71.463013,509.254066,7.072973,3704.0,57.0,180.0,209.0,7.625000
1108,CCC(c1cccc(O)c1)C(C)CN(C)C,0,1,DILIst Classification Oral,DILI,KWTWDQCKEHXFFR-UHFFFAOYSA-N,KWTWDQCKEHXFFR,CCC(c1cccc(O)c1)C(C)C[NH+](C)C,0,1,...,0.000000,9.353834,47.590867,222.185241,5.554631,448.0,21.0,74.0,82.0,3.722222
1109,CC(N)Cc1ccccc1,0,1,DILIst Classification Oral,DILI,KWTSXDURSIMDCE-UHFFFAOYSA-N,KWTSXDURSIMDCE,CC([NH3+])Cc1ccccc1,0,1,...,0.000000,8.564268,38.519688,136.112076,5.671336,126.0,9.0,44.0,46.0,2.333333


In [105]:
test_data_features.to_csv("Test_data_DILIst.csv", index=False)

In [106]:
train_data_features.to_csv("Train_data_liv_tox.csv", index=False)

In [4]:
import pandas as pd

test_data_features= pd.read_csv("Test_data_DILIst.csv")
test_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,1,1,DILIst Classification Oral,DILI,AAKJLRGGTJKAMG-UHFFFAOYSA-N,AAKJLRGGTJKAMG,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,0,0,...,0.000000,10.116298,64.016835,393.168856,7.560940,2396.0,43.0,142.0,163.0,6.833333
1,COC1(NC(=O)CSC(F)F)C(=O)N2C(C(=O)O)=C(CSc3nnnn...,1,1,DILIst Classification,DILI,UHRBTBZOWWGKMK-UHFFFAOYSA-N,UHRBTBZOWWGKMK,COC1(NC(=O)CSC(F)F)C(=O)[NH+]2C(C(=O)[O-])=C(C...,0,1,...,6.555357,10.733545,82.269503,496.064645,9.921293,3113.0,51.0,166.0,200.0,7.250000
2,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)N1CCCC1,1,1,DILIst Classification,DILI,UIEATEWHFDRYRU-UHFFFAOYSA-N,UIEATEWHFDRYRU,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)[NH+]1CCCC1,0,1,...,6.293419,9.845276,74.440588,367.274390,5.923780,1895.0,34.0,132.0,148.0,6.138889
3,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)...,1,1,DILIst Classification oral,DILI,UIOFUWFRIANQPC-UHFFFAOYSA-N,UIOFUWFRIANQPC,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)[NH+]2C1S...,0,0,...,7.853605,10.768843,83.380114,453.056148,9.639493,2402.0,53.0,170.0,211.0,6.291667
4,CC1OC1[P](=O)(=O)O,1,1,DILIst Classification Oral,DILI,UJNUDOLLRRCQDH-UHFFFAOYSA-N,UJNUDOLLRRCQDH,CC1OC1[P](=O)(=O)O,0,0,...,7.753194,9.180603,57.029675,137.000370,9.785741,61.0,7.0,42.0,48.0,1.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,1,DILIst Classification,DILI,KYRVNWMVYQXFEU-UHFFFAOYSA-N,KYRVNWMVYQXFEU,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,0,...,7.285507,9.858333,70.378545,301.052112,9.407879,994.0,29.0,110.0,129.0,4.666667
1107,CC1CNc2c(cccc2S(=O)(=O)NC(CCCN=C(N)N)C(=O)N2CC...,0,1,DILIst Classification Intravenous,DILI,KXNPVXPOPUZYGB-UHFFFAOYSA-N,KXNPVXPOPUZYGB,CC1CNc2c(cccc2S(=O)(=O)NC(CCC[NH+]=C(N)N)C(=O)...,0,1,...,0.000000,10.488075,71.463013,509.254066,7.072973,3704.0,57.0,180.0,209.0,7.625000
1108,CCC(c1cccc(O)c1)C(C)CN(C)C,0,1,DILIst Classification Oral,DILI,KWTWDQCKEHXFFR-UHFFFAOYSA-N,KWTWDQCKEHXFFR,CCC(c1cccc(O)c1)C(C)C[NH+](C)C,0,1,...,0.000000,9.353834,47.590867,222.185241,5.554631,448.0,21.0,74.0,82.0,3.722222
1109,CC(N)Cc1ccccc1,0,1,DILIst Classification Oral,DILI,KWTSXDURSIMDCE-UHFFFAOYSA-N,KWTSXDURSIMDCE,CC([NH3+])Cc1ccccc1,0,1,...,0.000000,8.564268,38.519688,136.112076,5.671336,126.0,9.0,44.0,46.0,2.333333


In [5]:
test_data_features.TOXICITY.value_counts()

1    716
0    395
Name: TOXICITY, dtype: int64

In [6]:
for i in test_data_features.Source_rank.unique():
    
    print(i)
    print(test_data_features[test_data_features["Source_rank"]==i]["TOXICITY"].value_counts())
    print(len(test_data_features[test_data_features["Source_rank"]==i]))

1
1    619
0    371
Name: TOXICITY, dtype: int64
990
13
1    97
0    24
Name: TOXICITY, dtype: int64
121


In [7]:
import pandas as pd

train_data_features= pd.read_csv("Train_data_liv_tox.csv")
train_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AHLBNYSZXLDEJQ-UHFFFAOYSA-N,AHLBNYSZXLDEJQ,CCCCCCCCCCCC(CC1OC(=O)C1CCCCCC)OC(=O)C(CC(C)C)...,0,1,...,0.000000,10.066499,70.246324,495.392374,5.629459,4600.0,41.0,154.0,167.0,8.500000
1,CCN(CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3ccc(F...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,ALVHFWNGOPERPY-UHFFFAOYSA-N,ALVHFWNGOPERPY,CC[NH+](CC)CC=NC(=O)c1c(C)[nH]c(Cc2c(O)[nH]c3c...,0,0,...,7.703008,10.247184,80.736166,399.219081,7.003844,2452.0,46.0,150.0,178.0,6.527778
2,CN=C(NC#N)NCCSCc1[nH]cnc1C,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AQIXAKUUQRKLND-UHFFFAOYSA-N,AQIXAKUUQRKLND,C[NH+]=C(NC#N)NCCSCc1[nH]cnc1C,0,0,...,6.529419,9.034200,61.513019,253.122992,7.444794,664.0,18.0,74.0,80.0,4.194444
3,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)O)C1OC1OC(C(=O)...,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,AVJBPWGFOQAPRH-UHFFFAOYSA-N,AVJBPWGFOQAPRH,CC(=O)NC1C(O)OC(CO)C(OS(=O)(=O)[O-])C1OC1OC(C(...,0,0,...,0.000000,10.438664,67.002785,475.064287,9.135852,2440.0,53.0,160.0,187.0,6.708333
4,O=C(O)c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,1,2,"Liver Toxicity Knowledge Base (LTKB), prescrip...",LivTox,BOFQWVMAQOTZIW-UHFFFAOYSA-N,BOFQWVMAQOTZIW,O=C([O-])c1ccc(-n2nc(-c3ccccc3O)nc2-c2ccccc2O)cc1,0,0,...,6.861711,10.291908,77.536823,372.098980,8.859500,1976.0,46.0,150.0,179.0,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16205,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZLHVIYHWWQYJID-UHFFFAOYSA-N,ZLHVIYHWWQYJID,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,0,...,0.000000,9.860110,51.351894,240.078644,8.002621,598.0,28.0,96.0,113.0,3.916667
16206,CCCCSP(=O)(SCCCC)SCCCC,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZOKXUAHZSKEQSS-UHFFFAOYSA-N,ZOKXUAHZSKEQSS,CCCCSP(=O)(SCCCC)SCCCC,0,0,...,0.000000,8.981304,47.595574,314.096165,7.138549,616.0,18.0,68.0,70.0,4.375000
16207,O=C1CC(c2ccccc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZONYXWQDUYMKFB-UHFFFAOYSA-N,ZONYXWQDUYMKFB,O=C1CC(c2ccccc2)Oc2ccccc21,0,0,...,0.000000,9.786336,50.012654,224.083730,7.727025,500.0,26.0,90.0,106.0,3.750000
16208,O=C(O)CCC(=O)c1ccc(-c2ccccc2)cc1,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZPAKPRAICRBAOD-UHFFFAOYSA-N,ZPAKPRAICRBAOD,O=C([O-])CCC(=O)c1ccc(-c2ccccc2)cc1,0,0,...,0.000000,9.556834,51.655875,253.087018,7.908969,808.0,25.0,92.0,103.0,4.305556


In [8]:
train_data = train_data_features[~train_data_features.Source_rank.isin([2, 4, 9, 10, 12])].reset_index(drop=True)
train_data

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CN(Cc1ccc(C(C)(C)C)cc1)Cc1cccc2ccccc12,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,ABJKWBDEJIDSJZ-UHFFFAOYSA-N,ABJKWBDEJIDSJZ,C[NH+](Cc1ccc(C(C)(C)C)cc1)Cc1cccc2ccccc12,0,0,...,0.000000,10.085517,58.622510,318.221626,6.119647,1498.0,36.0,126.0,145.0,5.138889
1,COc1cc2c(cc1OC)CC(=O)N(CCCN(C)CC1Cc3cc(OC)c(OC...,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,ACRHBAYQBXXRTO-UHFFFAOYSA-N,ACRHBAYQBXXRTO,COc1cc2c(cc1OC)CC(=O)[NH+](CCC[NH+](C)CC1Cc3cc...,0,0,...,5.293305,10.572726,78.667003,470.276975,6.531625,4157.0,58.0,178.0,212.0,7.750000
2,O=C(O)C(S)C(S)C(=O)O,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,ACTRVOBWPAIOHC-UHFFFAOYSA-N,ACTRVOBWPAIOHC,O=C([O-])C(S)C(S)C(=O)[O-],0,1,...,0.000000,8.702676,38.654211,179.956198,12.854014,121.0,12.0,42.0,45.0,2.333333
3,NC1CC1c1ccccc1,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,AELCINSCMGFISI-UHFFFAOYSA-N,AELCINSCMGFISI,[NH3+]C1CC1c1ccccc1,0,0,...,7.664347,9.206634,59.600240,134.096426,6.095292,119.0,10.0,52.0,61.0,2.222222
4,CC(C)(C)CC(=O)OCC(=O)C1(O)CCC2C3CC=C4CC(=O)C=C...,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,AEQGYUJKXATLBW-UHFFFAOYSA-N,AEQGYUJKXATLBW,CC(C)(C)CC(=O)OCC(=O)C1(O)CCC2C3CC=C4CC(=O)C=C...,0,0,...,7.247793,10.915325,85.019190,458.266839,6.454463,3111.0,67.0,190.0,235.0,6.729167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13698,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZLHVIYHWWQYJID-UHFFFAOYSA-N,ZLHVIYHWWQYJID,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,0,...,0.000000,9.860110,51.351894,240.078644,8.002621,598.0,28.0,96.0,113.0,3.916667
13699,CCCCSP(=O)(SCCCC)SCCCC,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZOKXUAHZSKEQSS-UHFFFAOYSA-N,ZOKXUAHZSKEQSS,CCCCSP(=O)(SCCCC)SCCCC,0,0,...,0.000000,8.981304,47.595574,314.096165,7.138549,616.0,18.0,68.0,70.0,4.375000
13700,O=C1CC(c2ccccc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZONYXWQDUYMKFB-UHFFFAOYSA-N,ZONYXWQDUYMKFB,O=C1CC(c2ccccc2)Oc2ccccc21,0,0,...,0.000000,9.786336,50.012654,224.083730,7.727025,500.0,26.0,90.0,106.0,3.750000
13701,O=C(O)CCC(=O)c1ccc(-c2ccccc2)cc1,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZPAKPRAICRBAOD-UHFFFAOYSA-N,ZPAKPRAICRBAOD,O=C([O-])CCC(=O)c1ccc(-c2ccccc2)cc1,0,0,...,0.000000,9.556834,51.655875,253.087018,7.908969,808.0,25.0,92.0,103.0,4.305556


In [9]:
train_data = train_data[~train_data.InChIKey14.isin(test_data_features.InChIKey14.to_list())].reset_index(drop=True)
train_data

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CN(Cc1ccc(C(C)(C)C)cc1)Cc1cccc2ccccc12,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,ABJKWBDEJIDSJZ-UHFFFAOYSA-N,ABJKWBDEJIDSJZ,C[NH+](Cc1ccc(C(C)(C)C)cc1)Cc1cccc2ccccc12,0,0,...,0.000000,10.085517,58.622510,318.221626,6.119647,1498.0,36.0,126.0,145.0,5.138889
1,COc1cc2c(cc1OC)CC(=O)N(CCCN(C)CC1Cc3cc(OC)c(OC...,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,ACRHBAYQBXXRTO-UHFFFAOYSA-N,ACRHBAYQBXXRTO,COc1cc2c(cc1OC)CC(=O)[NH+](CCC[NH+](C)CC1Cc3cc...,0,0,...,5.293305,10.572726,78.667003,470.276975,6.531625,4157.0,58.0,178.0,212.0,7.750000
2,CC(C)(C)CC(=O)OCC(=O)C1(O)CCC2C3CC=C4CC(=O)C=C...,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,AEQGYUJKXATLBW-UHFFFAOYSA-N,AEQGYUJKXATLBW,CC(C)(C)CC(=O)OCC(=O)C1(O)CCC2C3CC=C4CC(=O)C=C...,0,0,...,7.247793,10.915325,85.019190,458.266839,6.454463,3111.0,67.0,190.0,235.0,6.729167
3,CNc1ccccc1,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,AFBPFSWMIHJQDM-UHFFFAOYSA-N,AFBPFSWMIHJQDM,CNc1ccccc1,0,0,...,0.000000,8.298291,35.247635,107.073499,6.298441,64.0,7.0,34.0,36.0,2.000000
4,CCC1NC(=O)c2cc(S(N)(=O)=O)c(Cl)cc2N1,1,3,"human hepatotoxicity, hepatobiallry, literaty...",LivTox,AGMMTXLNIQSRCG-UHFFFAOYSA-N,AGMMTXLNIQSRCG,CCC1NC(=O)c2cc(S(N)(=O)=O)c(Cl)cc2[NH2+]1,0,0,...,0.000000,9.994653,51.598416,290.036066,9.356002,567.0,30.0,96.0,113.0,3.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12128,CC(C)C(Oc1ccc(CNC(=O)C2CCN(C(=O)CC(N)Cc3c(F)c(...,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZFLVPOVMZBOEHJ-UHFFFAOYSA-N,ZFLVPOVMZBOEHJ,CC(C)C(Oc1ccc(CNC(=O)C2CC[NH+](C(=O)CC([NH3+])...,0,1,...,6.605298,10.533029,90.548061,572.217839,8.059406,6887.0,65.0,204.0,238.0,8.777778
12129,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZLHVIYHWWQYJID-UHFFFAOYSA-N,ZLHVIYHWWQYJID,O=C1CC(c2ccc(O)cc2)Oc2ccccc21,0,0,...,0.000000,9.860110,51.351894,240.078644,8.002621,598.0,28.0,96.0,113.0,3.916667
12130,CCCCSP(=O)(SCCCC)SCCCC,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZOKXUAHZSKEQSS-UHFFFAOYSA-N,ZOKXUAHZSKEQSS,CCCCSP(=O)(SCCCC)SCCCC,0,0,...,0.000000,8.981304,47.595574,314.096165,7.138549,616.0,18.0,68.0,70.0,4.375000
12131,O=C1CC(c2ccccc2)Oc2ccccc21,0,16,Reactive Metabolite Mazzolari et al,LivTox,ZONYXWQDUYMKFB-UHFFFAOYSA-N,ZONYXWQDUYMKFB,O=C1CC(c2ccccc2)Oc2ccccc21,0,0,...,0.000000,9.786336,50.012654,224.083730,7.727025,500.0,26.0,90.0,106.0,3.750000


In [10]:
import pandas as pd

In [17]:
for Source_rank in train_data.Source_rank.unique():
    print(Source_rank)
    print(len(train_data[train_data["Source_rank"] == Source_rank]))
    print(train_data[train_data["Source_rank"] == Source_rank].TOXICITY.value_counts())

3
1163
1    588
0    575
Name: TOXICITY, dtype: int64
5
542
0    358
1    184
Name: TOXICITY, dtype: int64
6
671
1    369
0    302
Name: TOXICITY, dtype: int64
7
2204
1    1642
0     562
Name: TOXICITY, dtype: int64
8
1106
0    724
1    382
Name: TOXICITY, dtype: int64
11
445
0    237
1    208
Name: TOXICITY, dtype: int64
14
446
1    240
0    206
Name: TOXICITY, dtype: int64
15
5239
0    4550
1     689
Name: TOXICITY, dtype: int64
16
317
0    236
1     81
Name: TOXICITY, dtype: int64


In [18]:
test_data_features

,smiles_r,TOXICITY,Source_rank,Source,Data,InChIKey,InChIKey14,protonated_smiles_r,Mfp0,Mfp1,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,1,1,DILIst Classification Oral,DILI,AAKJLRGGTJKAMG-UHFFFAOYSA-N,AAKJLRGGTJKAMG,C#Cc1cccc(N=c2[nH]cnc3cc(OCCOC)c(OCCOC)cc23)c1,0,0,...,0.000000,10.116298,64.016835,393.168856,7.560940,2396.0,43.0,142.0,163.0,6.833333
1,COC1(NC(=O)CSC(F)F)C(=O)N2C(C(=O)O)=C(CSc3nnnn...,1,1,DILIst Classification,DILI,UHRBTBZOWWGKMK-UHFFFAOYSA-N,UHRBTBZOWWGKMK,COC1(NC(=O)CSC(F)F)C(=O)[NH+]2C(C(=O)[O-])=C(C...,0,1,...,6.555357,10.733545,82.269503,496.064645,9.921293,3113.0,51.0,166.0,200.0,7.250000
2,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)N1CCCC1,1,1,DILIst Classification,DILI,UIEATEWHFDRYRU-UHFFFAOYSA-N,UIEATEWHFDRYRU,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)[NH+]1CCCC1,0,1,...,6.293419,9.845276,74.440588,367.274390,5.923780,1895.0,34.0,132.0,148.0,6.138889
3,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)...,1,1,DILIst Classification oral,DILI,UIOFUWFRIANQPC-UHFFFAOYSA-N,UIOFUWFRIANQPC,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC1C(=O)[NH+]2C1S...,0,0,...,7.853605,10.768843,83.380114,453.056148,9.639493,2402.0,53.0,170.0,211.0,6.291667
4,CC1OC1[P](=O)(=O)O,1,1,DILIst Classification Oral,DILI,UJNUDOLLRRCQDH-UHFFFAOYSA-N,UJNUDOLLRRCQDH,CC1OC1[P](=O)(=O)O,0,0,...,7.753194,9.180603,57.029675,137.000370,9.785741,61.0,7.0,42.0,48.0,1.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,1,DILIst Classification,DILI,KYRVNWMVYQXFEU-UHFFFAOYSA-N,KYRVNWMVYQXFEU,COC(=O)N=c1[nH]c2ccc(C(=O)c3cccs3)cc2[nH]1,0,0,...,7.285507,9.858333,70.378545,301.052112,9.407879,994.0,29.0,110.0,129.0,4.666667
1107,CC1CNc2c(cccc2S(=O)(=O)NC(CCCN=C(N)N)C(=O)N2CC...,0,1,DILIst Classification Intravenous,DILI,KXNPVXPOPUZYGB-UHFFFAOYSA-N,KXNPVXPOPUZYGB,CC1CNc2c(cccc2S(=O)(=O)NC(CCC[NH+]=C(N)N)C(=O)...,0,1,...,0.000000,10.488075,71.463013,509.254066,7.072973,3704.0,57.0,180.0,209.0,7.625000
1108,CCC(c1cccc(O)c1)C(C)CN(C)C,0,1,DILIst Classification Oral,DILI,KWTWDQCKEHXFFR-UHFFFAOYSA-N,KWTWDQCKEHXFFR,CCC(c1cccc(O)c1)C(C)C[NH+](C)C,0,1,...,0.000000,9.353834,47.590867,222.185241,5.554631,448.0,21.0,74.0,82.0,3.722222
1109,CC(N)Cc1ccccc1,0,1,DILIst Classification Oral,DILI,KWTSXDURSIMDCE-UHFFFAOYSA-N,KWTSXDURSIMDCE,CC([NH3+])Cc1ccccc1,0,1,...,0.000000,8.564268,38.519688,136.112076,5.671336,126.0,9.0,44.0,46.0,2.333333


In [19]:
for Source_rank in test_data_features.Source_rank.unique():
    print(Source_rank)
    print(len(test_data_features[test_data_features["Source_rank"] == Source_rank]))
    print(test_data_features[test_data_features["Source_rank"] == Source_rank].TOXICITY.value_counts())

1
990
1    619
0    371
Name: TOXICITY, dtype: int64
13
121
1    97
0    24
Name: TOXICITY, dtype: int64
